<a href="https://colab.research.google.com/github/shahidnagra/Minor-projects/blob/main/Perform_Sentiment_Analysis_using_Scikit_learn/Perform_Sentiment_Analysis_using_Scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

df = pd.read_csv('movie_data.csv')
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [5]:
df['review'][0]
#the last sentence says my vote is 7 which makes sense that classification above shows positive

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

### Bag of words

Below, we will call the fit_transform method on CountVectorizer. This will construct the vocabulary of the bag-of-words model and transform the following three sentences into sparse feature vectors:
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two


In [6]:
#Lyrics of bob marley song
#we want to change into sparse vector by using bag of words method
#input this document, transform it into numerical values

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining, the weather is sweet, and one and one is two'])

bag = count.fit_transform(docs)

In [7]:
#vocabulary is saved in python dictionary
#which maps the unique words in the docs to integer indeces

print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [8]:
print(bag.toarray())

#N.B. Term frequency is number a word occurs in a document (count vector)
# 'is' term frequency in 3rd sentence is 3
#TFidf = Term Frequency Inverse Document Frequency
# It is technique used to downweight those frequently occurring words in the feature vectors.

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Raw term frequencies: *tf (t,d)*—the number of times a term t occurs in a document *d*

###  Word relevancy using term frequency-inverse document frequency

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

where $n_d$ is the total number of documents, and df(d, t) is the number of documents d that contain the term t.

In [9]:
# tf-idf is used to downweight the most frequent occuring words
#nd is = 3 here
# denominator is 1 to ensure non zeor
#logarithm is used to ensure words are not given too much weights

from sklearn.feature_extraction.text import TfidfTransformer

np.set_printoptions(precision=2)   #so array looks non-messy

tfidf = TfidfTransformer(use_idf = True, #enable reweighing
                        norm = 'l2',   
                         #each output will have a unit norm, here sum of squares of vector elements = 1
                        smooth_idf = True   
                         #weights by adding 1 to the document frequencies as if seen once more so avoid error of division by 0
                        )

print(tfidf.fit_transform(bag).toarray())

# 'is' term frequency in 3rd sentence is deflated to 0.45 because repeated in sentence 1&2

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


The equations for the idf and tf-idf that are implemented in scikit-learn are:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$
The tf-idf equation that is implemented in scikit-learn is as follows:

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

### Data Preparation

In [10]:
df.loc[0, 'review'][-50:]
#the last 50 characters
#i have html, tags, commas and emojis might also be available

'is seven.<br /><br />Title (Brazil): Not Available'

In [11]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    #replace html tags with a empty string
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    #get text emojes like smiley face, disappointed, sad
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    #we will move the emojis to end of text review
    return text

In [12]:
preprocessor(df.loc[0, 'review'][-50:])
#we will notice now panctuation like full stop, colon is not included
#also html tags and whats in between them have also been removed

'is seven title brazil not available'

In [13]:
preprocessor("<,/a> this :) is a :( test :-) !")

' this is a test :) :( :)'

In [14]:
df['review'] = df['review'].apply(preprocessor)

###  Tokenization of documents

In [15]:
#stemming to remove derivations of common words to a base word like organize, organized, organizer, organization
# stemming chops words and remove derivations
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

In [16]:
#tokenize the text and split the sentence into words according to occurance
#stemming technique

def tokenizer(text):
    return text.split()

In [17]:
def tokenizer_porter(text):
    return[porter.stem(word) for word in text.split()]

In [18]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [19]:
tokenizer_porter('runners like running and thus they run')

#the ends of words have been stripped
#idiosyncrncy like thus is changed to thu

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and thus she always runs a lot')[-10:] if w not in stop]

#removed 'a', 'and'

['runner', 'like', 'run', 'thu', 'alway', 'run', 'lot']

###  Transform Text Data into TF-IDF Vectors

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                        lowercase = False,
                        preprocessor = None, #because we already did on our data
                        tokenizer = tokenizer_porter, #stemming function
                        use_idf = True,    #to downgrade according to frequency
                        norm = 'l2',
                        smooth_idf = True   #to avoid division by zero
                       )

#split to x, y components
y = df.sentiment.values   #numpy array
x = tfidf.fit_transform(df.review)

###  Document Classification using Logistic Regression

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y,
                                                    random_state = 1, #to get same as instructor
                                                    test_size = 0.5,
                                                    shuffle = False
                                                   )

In [24]:
import pickle
#to dump our model on the disk
from sklearn.linear_model import LogisticRegressionCV
#Log Regression has hyperparemters and instead of manually trying to fine tune it
#we can use estimator of sklearn CV
#The output of a logistic model can be interpreted as a probability.

#Cross validation can be used to:
#1- Tune Model Hyperparamters
#2- Assess model performance out of sample

clf = LogisticRegressionCV(cv = 5, #cross validation fold
                          scoring = 'accuracy',
                          random_state = 0,
                          n_jobs = -1, #to deticate all our CPU to solving this task
                          verbose = 3, #to see output while doing computations
                          max_iter = 300    #for safety of convergence
                          ).fit(X_train, Y_train)

saved_model = open('saved_model.sav', 'wb')  #write bytes to this file
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.7min finished


 ###  Model Evaluation

In [25]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))   #load from disk the saved model

In [26]:
saved_clf.score(X_test, Y_test)

0.89608